<a href="https://colab.research.google.com/github/Trading-com-Dados/tutoriais_publicos/blob/main/20240707_Stock_Screener_Python_Twitter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://tradingcomdados.com/images/logotipo/logotipo-trading-com-dados.svg" width="300" align="left"/>

# **Stock Screener com Python**

### **1. Bibliotecas utilizadas**

Para um Screening em tempo real intraday, a melhor biblioteca seria MetaTrader5 (integra o código com o software MetaTrader 5 - MT5).<p>
Como a maioria do público talvez não tenha acesso ao MT5, vamos fazer um modelo de Screening usando o Yahoo Finance, mas que tem um delay de alguns minutos e, nesse caso, não indicado para screening intraday.

In [ ]:
!pip install yfinance
!pip install ta

In [33]:
import pandas as pd
import requests
import numpy as np
import yfinance as yf

from datetime import datetime, timedelta
import pytz
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import ta as ta

Composição atualizada do IBOV

In [34]:
session = requests.Session()
session.verify = False

url1 = 'https://sistemaswebb3-listados.b3.com.br/indexProxy/indexCall/GetPortfolioDay/'
url2 = 'eyJsYW5ndWFnZSI6InB0LWJyIiwicGFnZU51bWJlciI6MSwicGFnZVNpemUiOjEyMCwiaW5kZXgiOiJJQk9WIiwic2VnbWVudCI6IjEifQ=='

response = session.get(url1 + url2)

dados = pd.DataFrame(response.json()["results"])


/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning:

Unverified HTTPS request is being made to host 'sistemaswebb3-listados.b3.com.br'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings



In [53]:
dados.head()

,segment,cod,asset,type,part,partAcum,theoricalQty
0,None,RRRP3,3R PETROLEUM,ON NM,"0,321",None,238.441.689
1,None,ALOS3,ALLOS,ON NM,"0,561",None,532.616.595
2,None,ALPA4,ALPARGATAS,PN N1,"0,075",None,166.362.038
3,None,ABEV3,AMBEV S/A,ON,"2,427",None,4.394.245.879
4,None,ARZZ3,AREZZO CO,ON NM,"0,156",None,62.305.891


Transformação em lista

In [35]:
ibov = list(dados.cod)

### **2. Organizando o dataset**

#### 2.1. Baixar dados de cotações

In [36]:
tickers_IBOV_yf = [i + '.SA' for i in ibov]

data_fim = datetime.now(pytz.timezone("America/Sao_Paulo"))
data_fim = data_fim.strftime('%Y-%m-%d')

data_inicio = datetime.now(pytz.timezone("America/Sao_Paulo")) - timedelta(days=10)
data_inicio = data_inicio.strftime('%Y-%m-%d')


lista_OHLC = []

for i in tickers_IBOV_yf:
  cotacoes_IBOV = yf.download([i], start=data_inicio, end=data_fim, interval = "5m", progress= False)
  cotacoes_IBOV['Ativo'] = i
  lista_OHLC.append(cotacoes_IBOV)

#### 2.2. Calcular indicadores de Análise Técnica

In [43]:
for i in range(0,len(lista_OHLC)):
  MM9p = ta.trend.SMAIndicator(lista_OHLC[i]['Close'], window=9)
  MM20p = ta.trend.SMAIndicator(lista_OHLC[i]['Close'], window=20)
  MM200p = ta.trend.SMAIndicator(lista_OHLC[i]['Close'], window=200)
  IFR14p = ta.momentum.RSIIndicator(close=lista_OHLC[i]['Close'], window=14)
  lista_OHLC[i]['SMA_9p'] = MM20p.sma_indicator()
  lista_OHLC[i]['SMA_20p'] = MM20p.sma_indicator()
  lista_OHLC[i]['SMA_200p'] = MM200p.sma_indicator()
  lista_OHLC[i]['IFR14p'] = IFR14p.rsi()

In [44]:
lista_OHLC[0]

,Open,High,Low,Close,Adj Close,Volume,Ativo,SMA_9p,SMA_20p,SMA_200p,IFR14p
Datetime,,,,,,,,,,,
2024-06-25 10:05:00-03:00,26.010000,26.020000,25.830000,25.930000,25.930000,21800,RRRP3.SA,NaN,NaN,NaN,NaN
2024-06-25 10:10:00-03:00,25.959999,26.059999,25.879999,26.059999,26.059999,42200,RRRP3.SA,NaN,NaN,NaN,NaN
2024-06-25 10:15:00-03:00,26.049999,26.049999,25.889999,25.920000,25.920000,14800,RRRP3.SA,NaN,NaN,NaN,NaN
2024-06-25 10:20:00-03:00,25.930000,26.010000,25.920000,26.010000,26.010000,13400,RRRP3.SA,NaN,NaN,NaN,NaN
2024-06-25 10:25:00-03:00,25.980000,26.010000,25.959999,26.010000,26.010000,7600,RRRP3.SA,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
2024-07-04 16:30:00-03:00,27.760000,27.799999,27.670000,27.719999,27.719999,122400,RRRP3.SA,27.7440,27.7440,28.01900,44.017832
2024-07-04 16:35:00-03:00,27.740000,27.820000,27.639999,27.760000,27.760000,79500,RRRP3.SA,27.7480,27.7480,28.01795,47.748776
2024-07-04 16:40:00-03:00,27.760000,27.760000,27.690001,27.709999,27.709999,42800,RRRP3.SA,27.7485,27.7485,28.01675,43.817684


## 2.3. Visualização gráficos candlesticks e indicadores

In [22]:
list(enumerate(tickers_IBOV_yf))

[(0, 'ABEV3.SA'),
 (1, 'ALOS3.SA'),
 (2, 'ALPA4.SA'),
 (3, 'ARZZ3.SA'),
 (4, 'ASAI3.SA'),
 (5, 'AZUL4.SA'),
 (6, 'B3SA3.SA'),
 (7, 'BBAS3.SA'),
 (8, 'BBDC3.SA'),
 (9, 'BBDC4.SA'),
 (10, 'BBSE3.SA'),
 (11, 'BEEF3.SA'),
 (12, 'BPAC11.SA'),
 (13, 'BRAP4.SA'),
 (14, 'BRFS3.SA'),
 (15, 'BRKM5.SA'),
 (16, 'CCRO3.SA'),
 (17, 'CIEL3.SA'),
 (18, 'CMIG4.SA'),
 (19, 'CMIN3.SA'),
 (20, 'COGN3.SA'),
 (21, 'CPFE3.SA'),
 (22, 'CPLE6.SA'),
 (23, 'CRFB3.SA'),
 (24, 'CSAN3.SA'),
 (25, 'CSNA3.SA'),
 (26, 'CVCB3.SA'),
 (27, 'CYRE3.SA'),
 (28, 'DXCO3.SA'),
 (29, 'EGIE3.SA'),
 (30, 'ELET3.SA'),
 (31, 'ELET6.SA'),
 (32, 'EMBR3.SA'),
 (33, 'ENEV3.SA'),
 (34, 'ENGI11.SA'),
 (35, 'EQTL3.SA'),
 (36, 'EZTC3.SA'),
 (37, 'FLRY3.SA'),
 (38, 'GGBR4.SA'),
 (39, 'GOAU4.SA'),
 (40, 'HAPV3.SA'),
 (41, 'HYPE3.SA'),
 (42, 'IGTI11.SA'),
 (43, 'IRBR3.SA'),
 (44, 'ITSA4.SA'),
 (45, 'ITUB4.SA'),
 (46, 'JBSS3.SA'),
 (47, 'KLBN11.SA'),
 (48, 'LREN3.SA'),
 (49, 'LWSA3.SA'),
 (50, 'MGLU3.SA'),
 (51, 'MRFG3.SA'),
 (52, 'MRVE3.SA'),

In [24]:
# Como exemplo, vamos produzir os gráficos de candlestick (com suas MMs), OBV e IFR do ticker 'MGLU3.SA' (número 59 na lista)

df_figure = lista_OHLC[59]

fig = make_subplots(rows=2, cols=1)

fig.add_trace(go.Candlestick(name='ABEV3', x=df_figure.index, open=df_figure['Open'], high = df_figure['High'], low=df_figure['Low'], close=df_figure['Close']), row=1,col=1)
fig.add_trace(go.Scatter(name='SMA_9p', x=df_figure.index, y=df_figure['SMA_9p'],marker_color='red'), row=1, col=1)
fig.add_trace(go.Scatter(name='SMA_20p', x=df_figure.index, y=df_figure['SMA_20p'],marker_color='blue'), row=1, col=1)
fig.add_trace(go.Scatter(name='SMA_200p', x=df_figure.index, y=df_figure['SMA_200p'],marker_color='black'), row=1, col=1)

fig.add_trace(go.Scatter(name='IFR14p', x=df_figure.index, y=df_figure['IFR14p'],marker_color='orange'), row=2, col=1)

fig.update_xaxes(title_text="<b> Data",rangebreaks=[dict(bounds=["sat", "mon"]),dict(bounds=[17, 10], pattern="hour")])

fig.update_layout(xaxis_rangeslider_visible=False, title_text='Price, OBV e IFR',width=1000,height=1000)
fig.show()

# 3. Screening dos ativos listados

Vamos determinar algumas regras e fazer a varredura de quantos e quais ativos se encaixam nesse momento. Estaremos buscando ativos que apresentaram:

* Regra 01: Fechamento com preço acima da MM20p e MM200p;
* Regra 02: MM9p > MM20p > MM200p;
* Regra 03: IFR14p < 50

In [42]:
lista_OHLC[0]

,Open,High,Low,Close,Adj Close,Volume,Ativo
Datetime,,,,,,,
2024-06-25 10:05:00-03:00,26.010000,26.020000,25.830000,25.930000,25.930000,21800,RRRP3.SA
2024-06-25 10:10:00-03:00,25.959999,26.059999,25.879999,26.059999,26.059999,42200,RRRP3.SA
2024-06-25 10:15:00-03:00,26.049999,26.049999,25.889999,25.920000,25.920000,14800,RRRP3.SA
2024-06-25 10:20:00-03:00,25.930000,26.010000,25.920000,26.010000,26.010000,13400,RRRP3.SA
2024-06-25 10:25:00-03:00,25.980000,26.010000,25.959999,26.010000,26.010000,7600,RRRP3.SA
...,...,...,...,...,...,...,...
2024-07-04 16:30:00-03:00,27.760000,27.799999,27.670000,27.719999,27.719999,122400,RRRP3.SA
2024-07-04 16:35:00-03:00,27.740000,27.820000,27.639999,27.760000,27.760000,79500,RRRP3.SA
2024-07-04 16:40:00-03:00,27.760000,27.760000,27.690001,27.709999,27.709999,42800,RRRP3.SA


Filtro de acordo com as regras estabelecidas

In [45]:
ls_screening = []

for i in range(0,len(lista_OHLC)):
    if (lista_OHLC[i].iloc[-1]['Close'] >= lista_OHLC[i].iloc[-1]['SMA_20p']) and
     (lista_OHLC[i].iloc[-1]['Close'] >= lista_OHLC[i].iloc[-1]['SMA_200p']) and
      (lista_OHLC[i].iloc[-1]['SMA_9p'] >= lista_OHLC[i].iloc[-1]['SMA_20p']) and
       (lista_OHLC[i].iloc[-1]['SMA_20p'] >= lista_OHLC[i].iloc[-1]['SMA_200p']) and
        (lista_OHLC[i].iloc[-1]['IFR14p'] < 50):
        ls_screening.append((tickers_IBOV_yf[i],i))

Apenas os ativos que atendem

In [46]:
ls_screening

[('B3SA3.SA', 7), ('HAPV3.SA', 41), ('UGPA3.SA', 78)]

Retornando ativos que atendem e não atendem

In [49]:
ls_screening = []

for i in range(0,len(lista_OHLC)):
  if (lista_OHLC[i].iloc[-1]['Close'] >= lista_OHLC[i].iloc[-1]['SMA_20p']) and (lista_OHLC[i].iloc[-1]['Close'] >= lista_OHLC[i].iloc[-1]['SMA_200p']) and (lista_OHLC[i].iloc[-1]['SMA_9p'] >= lista_OHLC[i].iloc[-1]['SMA_20p']) and (lista_OHLC[i].iloc[-1]['SMA_20p'] >= lista_OHLC[i].iloc[-1]['SMA_200p']) and (lista_OHLC[i].iloc[-1]['IFR14p'] < 50):
    ls_screening.append((tickers_IBOV_yf[i],i))
    print("O ativo indexado como",i,"("+tickers_IBOV_yf[i]+") atende a todas as regras estabelecidas")
  else:
    print(tickers_IBOV_yf[i],"não atende")

RRRP3.SA não atende
ALOS3.SA não atende
ALPA4.SA não atende
ABEV3.SA não atende
ARZZ3.SA não atende
ASAI3.SA não atende
AZUL4.SA não atende
O ativo indexado como 7 (B3SA3.SA) atende a todas as regras estabelecidas
BBSE3.SA não atende
BBDC3.SA não atende
BBDC4.SA não atende
BRAP4.SA não atende
BBAS3.SA não atende
BRKM5.SA não atende
BRFS3.SA não atende
BPAC11.SA não atende
CRFB3.SA não atende
CCRO3.SA não atende
CMIG4.SA não atende
CIEL3.SA não atende
COGN3.SA não atende
CPLE6.SA não atende
CSAN3.SA não atende
CPFE3.SA não atende
CMIN3.SA não atende
CVCB3.SA não atende
CYRE3.SA não atende
DXCO3.SA não atende
ELET3.SA não atende
ELET6.SA não atende
EMBR3.SA não atende
ENGI11.SA não atende
ENEV3.SA não atende
EGIE3.SA não atende
EQTL3.SA não atende
EZTC3.SA não atende
FLRY3.SA não atende
GGBR4.SA não atende
GOAU4.SA não atende
NTCO3.SA não atende
SOMA3.SA não atende
O ativo indexado como 41 (HAPV3.SA) atende a todas as regras estabelecidas
HYPE3.SA não atende
IGTI11.SA não atende
IRBR3.SA

Plotagem

In [52]:
# Checagem gráfica de um ativo que atende a todas as regras do Screening de Compra
# Cheque todas as regras nos gráficos e veja que realmente funcionou

df_figure = lista_OHLC[7]

fig = make_subplots(rows=2, cols=1)

fig.add_trace(go.Candlestick(name='B3SA3', x=df_figure.index, open=df_figure['Open'], high = df_figure['High'], low=df_figure['Low'], close=df_figure['Close']), row=1,col=1)
fig.add_trace(go.Scatter(name='SMA_9p', x=df_figure.index, y=df_figure['SMA_9p'],marker_color='red'), row=1, col=1)
fig.add_trace(go.Scatter(name='SMA_20p', x=df_figure.index, y=df_figure['SMA_20p'],marker_color='blue'), row=1, col=1)
fig.add_trace(go.Scatter(name='SMA_200p', x=df_figure.index, y=df_figure['SMA_200p'],marker_color='black'), row=1, col=1)

fig.add_trace(go.Scatter(name='IFR14p', x=df_figure.index, y=df_figure['IFR14p'],marker_color='orange'), row=2, col=1)

fig.update_xaxes(title_text="<b> Data",rangebreaks=[dict(bounds=["sat", "mon"]),dict(bounds=[17, 10], pattern="hour")])

fig.update_layout(xaxis_rangeslider_visible=False, title_text='Price, OBV e IFR',width=1000,height=1000)
fig.show()